In [ ]:
import opendatasets as od
import pandas
 
od.download("https://www.kaggle.com/competitions/feather-in-focus/data")

In [32]:
#https://www.kaggle.com/code/chekoduadarsh/starters-guide-convolutional-xgboost/notebook
#https://www.kaggle.com/code/pedrolucasbritodes/bird-image-classification-cnn-89-accuracy/notebook
#https://stats.stackexchange.com/questions/404809/is-it-advisable-to-use-output-from-a-ml-model-as-a-feature-in-another-ml-model

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

#how many epchos still revert
patience1 = 3

base_learning_rate = 0.001 
#number before fine_tuning
number_epcho1 = 20
#number after fine_tuning
number_epcho2 = 20
number_class = 200
#needs to be the same number of classes
output_Neurons = 200
Neurons = 512
batch_size = 32
drop_out_rate = 0.3
image_shape = (224,224)
image_shape_full = (224,224,3)
current_directory = "feather-in-focus"

In [33]:
#loading files 
class_names2 = np.load('feather-in-focus/class_names.npy', allow_pickle=True).item()

#load all files because why not
test_image_path = pd.read_csv('feather-in-focus/test_images_path.csv')
test_image_path['total_path'] = current_directory + "/test_images" + test_image_path['image_path'] 
test_image_path['total_path'] = test_image_path['total_path'].str.replace("\\", "/" )
test_image = pd.read_csv('feather-in-focus/test_images_sample.csv')


#importing training data 
train_image = pd.read_csv('feather-in-focus/train_images.csv')
train_image['total_path'] = current_directory + "/train_images" + train_image['image_path']
train_image['total_path'] = train_image['total_path'].str.replace("\\", "/" )
#train_image['label'] = train_image['label'].apply(str)
train_image.head()

#importing bird data 
bird_data = pd.read_csv('output_birds.csv')


bird_data2 = train_image.merge(bird_data, how = 'left', left_on = 'label', right_on = 'bird_number_x')
bird_data2['label'] = bird_data2['label'].apply(str)

bird_data2 = bird_data2[['image_path', 'label', 'total_path', 'bird_class', 'Wing.Length',
                        'Tail.Length', 'Beak.Width', 'Beak.Depth', 'Tarsus.Length','Mass']]

#bird_data2

/scratch-local/scur2079.4593057/ipykernel_756748/3711254665.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test_image_path['total_path'] = test_image_path['total_path'].str.replace("\\", "/" )
/scratch-local/scur2079.4593057/ipykernel_756748/3711254665.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train_image['total_path'] = train_image['total_path'].str.replace("\\", "/" )


In [34]:
#Wing_min = bird_data2['Wing.Length'].min()
#Wing_max = bird_data2['Wing.Length'].max()
#bird_data2['Wing.Length'] =  (bird_data2['Wing.Length'] - Wing_min) / (Wing_max - Wing_min)

Wing_mean = bird_data2['Wing.Length'].mean()
Wing_std = bird_data2['Wing.Length'].std()
bird_data2['Wing.Length'] =  (bird_data2['Wing.Length'] - Wing_mean) / (Wing_std)


#tail_min = bird_data2['Tail.Length'].min()
#tail_max = bird_data2['Tail.Length'].max()
tail_mean = bird_data2['Tail.Length'].mean()
tail_std = bird_data2['Tail.Length'].std()

bird_data2['Tail.Length'] = (bird_data2['Tail.Length'] - tail_mean ) / tail_std 

#bird_data2['Tail.Length'] = (bird_data2['Tail.Length'] - tail_min ) / (tail_max - tail_min) 
#mass_min = bird_data2['Mass'].min()
#mass_max = bird_data2['Mass'].max()
mass_mean = bird_data2['Beak.Width'].mean()
mass_std = bird_data2['Beak.Width'].std()
bird_data2['Beak.Width'] = (bird_data2['Beak.Width'] - mass_mean) / mass_std
#bird_data2['Mass'] = (bird_data2['Mass'] - mass_min) / (mass_max - mass_min)


bird_data2 = bird_data2[['total_path','image_path','Wing.Length','Tail.Length','Beak.Width','label']]
bird_data2

,total_path,image_path,Wing.Length,Tail.Length,Beak.Width,label
0,feather-in-focus/train_images/train_images/1.jpg,/train_images/1.jpg,2.587659,0.761151,2.767893,1
1,feather-in-focus/train_images/train_images/2.jpg,/train_images/2.jpg,2.587659,0.761151,2.767893,1
2,feather-in-focus/train_images/train_images/3.jpg,/train_images/3.jpg,2.587659,0.761151,2.767893,1
3,feather-in-focus/train_images/train_images/4.jpg,/train_images/4.jpg,2.587659,0.761151,2.767893,1
4,feather-in-focus/train_images/train_images/5.jpg,/train_images/5.jpg,2.587659,0.761151,2.767893,1
...,...,...,...,...,...,...
3921,feather-in-focus/train_images/train_images/392...,/train_images/3922.jpg,-0.836685,-0.762179,-0.767175,200
3922,feather-in-focus/train_images/train_images/392...,/train_images/3923.jpg,-0.836685,-0.762179,-0.767175,200
3923,feather-in-focus/train_images/train_images/392...,/train_images/3924.jpg,-0.836685,-0.762179,-0.767175,200
3924,feather-in-focus/train_images/train_images/392...,/train_images/3925.jpg,-0.836685,-0.762179,-0.767175,200


In [35]:
# we need to add a k-folds, i have stratified so it works a bit better
train, valid = train_test_split(bird_data2, 
                    test_size=0.2,
                    stratify = train_image['label'],
                    random_state=420)


In [36]:
# this creates the type of augmentation that is done! 
datagen = ImageDataGenerator(
    #rotation_range=15,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 15,
    horizontal_flip = True,
    zoom_range = 0.20)
#look at aug from group chat 

#okay this method seems WAY better. only ever use aug images
train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    x_col='total_path',
    y_col=['Wing.Length','Tail.Length','Beak.Width'],
           # Assuming 'label' contains the continuous values
    color_mode='rgb',
    target_size=image_shape,
    class_mode='raw',  # Use 'raw' for continuous values instead of 'categorical'
    batch_size=batch_size  # Define your batch size
)


valid_generator=datagen.flow_from_dataframe(
dataframe=valid,
x_col='total_path',
y_col=['Wing.Length','Tail.Length','Beak.Width'],  # Assuming 'label' contains the continuous values
color_mode='rgb',
target_size=image_shape,
class_mode='raw',  # Use 'raw' for continuous values instead of 'categorical'
batch_size=batch_size  # Define your batch size
)



Found 3140 validated image filenames.
Found 786 validated image filenames.


In [37]:
from keras.models import Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
import tensorflow as tf
IM_WIDTH, IM_HEIGHT =  224, 224
class multi_ouput_bird():
    """
    Used to generate our multi-output model. This CNN contains three branches, one for age, other for 
    sex and another for race. Each branch contains a sequence of Convolutional Layers that is defined
    on the make_default_hidden_layers method.
    """
    def make_default_hidden_layers(self, inputs):
        """
        Used to generate a default set of hidden layers. The structure used in this network is defined as:
        
        Conv2D -> BatchNormalization -> Pooling -> Dropout
        """
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        return x
    def tail_length_branch(self, inputs):   
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        # changed from linear to stop ngative 
        x = Activation("relu", name="tail_length_output")(x)
        return x
    def mass_branch(self, inputs):   
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
                # changed from linear to stop ngative
        x = Activation("relu", name="mass_output")(x)
        return x
    def wing_length_branch(self, inputs):   
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks, 
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
                # changed from linear to stop ngative
        x = Activation("relu", name="wing_length_output")(x)
        return x
    
    
    
    def assemble_full_model(self, width, height):
        """
        Used to assemble our multi-output model CNN.
        """
        input_shape = (height, width, 3)
        inputs = Input(shape=input_shape)
        wing_length = self.wing_length_branch(inputs)
        mass = self.mass_branch(inputs)
        tail_length = self.tail_length_branch(inputs)
        model = Model(inputs=inputs, outputs = [wing_length, mass, tail_length], name="face_net")
        return model
    
model = multi_ouput_bird().assemble_full_model(IM_WIDTH, IM_HEIGHT)
                                              

In [38]:
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D, Input

IM_WIDTH, IM_HEIGHT = 224, 224

class MultiOutputBird():
    def tail_length_branch(self, inputs):   
        x = Dense(500, activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.8)(x)
        x = Dense(1, activation='linear', name="tail_length_output")(x)
        return x

    def mass_branch(self, inputs):   
        x = Dense(500, activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.8)(x)
        x = Dense(1, activation='linear', name="mass_output")(x)
        return x

    def wing_length_branch(self, inputs):   
        x = Dense(500, activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.8)(x)
        x = Dense(1, activation='linear', name="wing_length_output")(x)
        return x
    
    def assemble_full_model(self, width, height):
        input_shape = (height, width, 3)
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        
        for layer in base_model.layers:
            layer.trainable = False

        x = base_model.output
        x = GlobalAveragePooling2D()(x)

        wing_length = self.wing_length_branch(x)
        mass = self.mass_branch(x)
        tail_length = self.tail_length_branch(x)

        model = Model(inputs=base_model.input, outputs=[wing_length, mass, tail_length], name="face_net")
        return model
    

model_builder = MultiOutputBird()
model = model_builder.assemble_full_model(IM_WIDTH, IM_HEIGHT)


In [39]:
from keras.optimizers import Adam
init_lr = 1e-4
epochs = 15
opt = Adam(lr=init_lr, decay=init_lr / epochs)

model.compile(optimizer=opt,
              loss={
                  'tail_length_output': 'mse',
                  'mass_output': 'mse',
                  'wing_length_output': 'mse'},
              metrics={
                  'tail_length_output': 'mae',
                  'mass_output': 'mae',
                  'wing_length_output': 'mae'})


/sw/arch/RHEL8/EB_production/2022/software/TensorFlow/2.11.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [40]:
from keras.callbacks import ModelCheckpoint
batch_size = 10
valid_batch_size = 10
#train_gen = data_generator.generate_images(train_id, is_training=True, batch_size=batch_size)
#valid_gen = data_generator.generate_images(valid_id, is_training=True, batch_size=valid_batch_size)

early_stop = tf.keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,)

#callbacks = [
#    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
#]
history = model.fit(train_generator,
                    steps_per_epoch=80,
                    epochs=30,
                    callbacks=[early_stop],
                    validation_data=valid_generator)

Epoch 1/30
80/80 [==============================] - 42s 468ms/step - loss: 26.9511 - wing_length_output_loss: 9.1343 - mass_output_loss: 8.6831 - tail_length_output_loss: 9.1337 - wing_length_output_mae: 2.3832 - mass_output_mae: 2.3183 - tail_length_output_mae: 2.3666 - val_loss: 3.8997 - val_wing_length_output_loss: 1.4061 - val_mass_output_loss: 1.3339 - val_tail_length_output_loss: 1.1598 - val_wing_length_output_mae: 0.8673 - val_mass_output_mae: 0.8128 - val_tail_length_output_mae: 0.7961
Epoch 2/30
80/80 [==============================] - 35s 444ms/step - loss: 23.3181 - wing_length_output_loss: 7.8152 - mass_output_loss: 7.7089 - tail_length_output_loss: 7.7940 - wing_length_output_mae: 2.1975 - mass_output_mae: 2.1926 - tail_length_output_mae: 2.2222 - val_loss: 4.0175 - val_wing_length_output_loss: 1.2502 - val_mass_output_loss: 1.3528 - val_tail_length_output_loss: 1.4145 - val_wing_length_output_mae: 0.8332 - val_mass_output_mae: 0.8565 - val_tail_length_output_mae: 0.8757


In [41]:

current_directory = "feather-in-focus"
model.save('{}/keras_final_continous.keras'.format(current_directory))


# image folder
folder_path = '{}/test_images/test_images/'.format(current_directory)
# path to model
model_path = '{}/keras_final_continous.keras'.format(current_directory)
# dimensions of images
img_width, img_height = 224, 224



In [42]:
from keras.models import load_model
import keras.utils as image
# load the trained model
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
model_path = '{}/keras_final_continous.keras'.format(current_directory)
model = load_model(model_path)
model.compile(loss={
                  'tail_length_output': 'mse',
                  'mass_output': 'mse',
                  'wing_length_output': 'mse'},
    optimizer=adam_optimizer,
    metrics=["accuracy"])

In [44]:
#for layers in model.layers:
 #   if "BatchNormalization" in layers.__class__.__name__:
        #layers.trainable = False
       # print(layers.weights)
        

In [45]:
name = list()
images = []
folder_path = 'feather-in-focus/train_images/train_images'  
#folder_path = 'feather-in-focus/test_images/test_images' 
for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)
    # Check if the item is a file (not a directory)
    if os.path.isfile(img_path):
        name.append(img_name)
        img = image.load_img(img_path, target_size=image_shape)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        images.append(img)
        # Process the image further as needed
    else:
        # Skip directories
        continue
        
# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict(images, batch_size=10)
#classes_x=np.argmax(classes,axis=1)

393/393 [==============================] - 4s 8ms/step


In [46]:
final_df = pd.DataFrame(
    {'id': name,
     'label1': classes[0].tolist(),
     'label2': classes[1].tolist(),
     'label3': classes[2].tolist(),
    })
final_df['label1'] = final_df['label1'].apply(lambda x: float(x[0]))
final_df['label2'] = final_df['label2'].apply(lambda x: float(x[0]))
final_df['label3'] = final_df['label3'].apply(lambda x: float(x[0]))
final_df

,id,label1,label2,label3
0,2374.jpg,0.638535,0.286833,-0.546973
1,1135.jpg,-0.359460,-0.128332,-0.157745
2,2777.jpg,0.237911,-0.083777,-0.217412
3,3780.jpg,0.099687,-0.294841,-0.101249
4,1538.jpg,-0.676401,-0.023473,-0.348345
...,...,...,...,...
3921,1412.jpg,-0.613986,-0.293242,-0.654074
3922,144.jpg,-0.057646,-0.031628,0.026038
3923,1815.jpg,0.563421,0.700343,0.850344
3924,682.jpg,0.767546,1.189890,0.830227


In [47]:
final_df['id2'] = "/train_images/"+final_df['id']

In [48]:
final_df2 = final_df.merge(train, left_on = 'id2', right_on = 'image_path')
final_df2

,id,label1,label2,label3,id2,total_path,image_path,Wing.Length,Tail.Length,Beak.Width,label
0,2374.jpg,0.638535,0.286833,-0.546973,/train_images/2374.jpg,feather-in-focus/train_images/train_images/237...,/train_images/2374.jpg,1.085597,-0.779769,0.133588,86
1,1135.jpg,-0.359460,-0.128332,-0.157745,/train_images/1135.jpg,feather-in-focus/train_images/train_images/113...,/train_images/1135.jpg,-0.478238,-0.081430,0.167579,38
2,2777.jpg,0.237911,-0.083777,-0.217412,/train_images/2777.jpg,feather-in-focus/train_images/train_images/277...,/train_images/2777.jpg,-0.026319,0.377680,-0.631211,105
3,3780.jpg,0.099687,-0.294841,-0.101249,/train_images/3780.jpg,feather-in-focus/train_images/train_images/378...,/train_images/3780.jpg,-0.699321,-0.758661,-0.699193,178
4,1538.jpg,-0.676401,-0.023473,-0.348345,/train_images/1538.jpg,feather-in-focus/train_images/train_images/153...,/train_images/1538.jpg,-0.260407,-1.057698,-0.359282,52
...,...,...,...,...,...,...,...,...,...,...,...
3135,1648.jpg,-0.517227,-0.321531,-0.284659,/train_images/1648.jpg,feather-in-focus/train_images/train_images/164...,/train_images/1648.jpg,-0.393707,-0.134201,0.184574,56
3136,2651.jpg,-0.366057,-0.207045,-0.463958,/train_images/2651.jpg,feather-in-focus/train_images/train_images/265...,/train_images/2651.jpg,-0.686316,-0.744588,-0.665202,99
3137,1412.jpg,-0.613986,-0.293242,-0.654074,/train_images/1412.jpg,feather-in-focus/train_images/train_images/141...,/train_images/1412.jpg,-0.652178,-0.776251,-0.342287,48
3138,144.jpg,-0.057646,-0.031628,0.026038,/train_images/144.jpg,feather-in-focus/train_images/train_images/144...,/train_images/144.jpg,-0.148240,-0.962710,-0.240314,5


In [50]:
final_df3 = final_df2[['id', 'label1', 'label2', 'label3', 'Wing.Length', 'Tail.Length', 'Beak.Width']]
final_df3

,id,label1,label2,label3,Wing.Length,Tail.Length,Beak.Width
0,2374.jpg,0.638535,0.286833,-0.546973,1.085597,-0.779769,0.133588
1,1135.jpg,-0.359460,-0.128332,-0.157745,-0.478238,-0.081430,0.167579
2,2777.jpg,0.237911,-0.083777,-0.217412,-0.026319,0.377680,-0.631211
3,3780.jpg,0.099687,-0.294841,-0.101249,-0.699321,-0.758661,-0.699193
4,1538.jpg,-0.676401,-0.023473,-0.348345,-0.260407,-1.057698,-0.359282
...,...,...,...,...,...,...,...
3135,1648.jpg,-0.517227,-0.321531,-0.284659,-0.393707,-0.134201,0.184574
3136,2651.jpg,-0.366057,-0.207045,-0.463958,-0.686316,-0.744588,-0.665202
3137,1412.jpg,-0.613986,-0.293242,-0.654074,-0.652178,-0.776251,-0.342287
3138,144.jpg,-0.057646,-0.031628,0.026038,-0.148240,-0.962710,-0.240314


In [51]:
final_df3.corr()

,label1,label2,label3,Wing.Length,Tail.Length,Beak.Width
label1,1.000000,0.854963,0.848891,0.767470,0.588240,0.574518
label2,0.854963,1.000000,0.853405,0.767543,0.595019,0.578904
label3,0.848891,0.853405,1.000000,0.770625,0.602210,0.566734
Wing.Length,0.767470,0.767543,0.770625,1.000000,0.734509,0.730497
Tail.Length,0.588240,0.595019,0.602210,0.734509,1.000000,0.503263
Beak.Width,0.574518,0.578904,0.566734,0.730497,0.503263,1.000000


In [31]:
idea = final_df3[final_df3['label2'] != 0]
idea

,id,label1,label2,label3,Wing.Length,Tail.Length,Mass
0,2374.jpg,1.357677,0.397800,0.590599,1.085597,-0.779769,2.050542
1,1135.jpg,-0.369133,-0.080345,-0.441112,-0.478238,-0.081430,-0.370587
2,2777.jpg,-0.465114,-0.365794,-0.668487,-0.026319,0.377680,-0.347456
3,3780.jpg,-0.363806,-0.361650,-0.420742,-0.699321,-0.758661,-0.384989
4,1538.jpg,-0.511576,-0.367471,-0.395818,-0.260407,-1.057698,0.043809
...,...,...,...,...,...,...,...
3135,1648.jpg,-0.210404,-0.290074,0.045095,-0.393707,-0.134201,-0.344073
3136,2651.jpg,-0.461339,-0.313309,-0.558844,-0.686316,-0.744588,-0.385098
3137,1412.jpg,-0.521583,-0.438476,-0.234674,-0.652178,-0.776251,-0.388153
3138,144.jpg,0.320526,0.082603,0.016675,-0.148240,-0.962710,-0.122582


In [ ]:
#blue = classes[0].tolist() *(np.max(train['Wing.Length']) - np.min(train['Wing.Length'])) +  np.min(train['Wing.Length'])
# Perform operations step by step
#blue_values = np.array(classes[2].tolist())
#min_wing_length = np.min(train['Wing.Length'])
#max_wing_length = np.max(train['Wing.Length'])

blue_values = np.array(classes[0].tolist())
mean_wing_length = np.mean(train['Wing.Length'])
std_wing_length = np.std(train['Wing.Length'])

#result = (blue_values * (max_wing_length - min_wing_length)) + min_wing_length
result = blue_values*std_wing_length + mean_wing_length

np.mean(result)


In [ ]:
mean_wing_length

In [ ]:

final_df['id2'] = "/test_images/"+final_df['id']
final_df['label_predict'] = final_df['label_predict'].astype(int)
final_df = final_df[['label_predict','id2']]

idea = np.sort(train_image['label'].unique())
idea2 = pd.DataFrame(idea)
idea2['index1'] = idea2.index
idea2.columns = ['values', 'index1']
idea2['values'] = idea2['values'].astype(int)

final_df4 = final_df.merge(test_image_path[['image_path','id']], left_on = 'id2', right_on='image_path')

final_df4 = final_df4[['id','label_predict']]

final_df2 = final_df4.merge(idea2, how = 'left', left_on = 'label_predict', right_on = 'index1')
final_df3 = final_df2[['id','values']]
#two issues the class values arent right
final_df3.columns = ['id','label']
final_df3


In [ ]:
final_df3.to_csv('output_folder/output.csv', index=False)

In [ ]:
import json

# Define the metadata for your dataset
metadata = {
    "title": "output.csv",
    "id": "uvajonathon",
    "licenses": [
        {
            "name": "License Name",
            "id": "uvajonathon",
            "url": "URL to License if applicable"
        }
    ],
    "description": "Description of your dataset",
    "keywords": ["keyword1", "keyword2"],
    "language": "en",
    "isPrivate": False,  # Set to True or False based on privacy
    "collaborators": ["uvajonathon"],
    "data": [
        {
            "description": "Description of the data",
            "name": "output.csv"
        }
    ]
}

# Save the metadata to dataset-metadata.json
file_path = 'output_folder/dataset-metadata.json'  # Specify the path where you want to save the file

with open(file_path, 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

print(f"Dataset metadata saved to {file_path}")


In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
# Set the path to your Kaggle API credentials file (kaggle.json)
os.environ['KAGGLE_CONFIG_DIR'] = '/gpfs/home2/scur2079/'
# Instantiate the Kaggle API
api = KaggleApi()
# Replace 'competition-name' with the competition name (as shown in the competition URL)
competition_name = 'feather-in-focus'
# Replace 'file-path.csv' with the path to your CSV file
file_path = '/gpfs/home2/scur2079/output_folder'

In [ ]:

# Specify the competition to which you want to upload the file
api.competition_name = competition_name
# Upload the CSV file to the competition
api.dataset_create_new(file_path)